In [2]:
import pandas as pd
from fredapi import Fred
import sys
import os

# Add config module to sys path
sys.path.insert(0, os.path.abspath('../../config'))

# Import API key
from api_keys import fred_api_key


In [3]:
#Set up API Key

fred = Fred(api_key=f'{fred_api_key}')

In [4]:
# Retrieve 10 Years Treasury security rates (monthly) from FRED API

data_10year_treasury = fred.get_series('GS10')

In [5]:
# Retain only data since Jan 1959. 

data_10year_treasury = data_10year_treasury.loc[data_10year_treasury.index > '1958-12-31']

In [6]:
# View 10 year series

data_10year_treasury.head(5)

1959-01-01    4.02
1959-02-01    3.96
1959-03-01    3.99
1959-04-01    4.12
1959-05-01    4.31
dtype: float64

In [7]:
# Retrieve 2 Years Treasury security rates (monthly) from FRED API

data_2year_treasury = fred.get_series('GS2')

In [8]:
# View 2 years series

data_2year_treasury.head()

1976-06-01    7.06
1976-07-01    6.85
1976-08-01    6.63
1976-09-01    6.42
1976-10-01    5.98
dtype: float64

In [9]:
# Retrieve 1 Year Treasury security rates (monthly) from FRED API

data_1year_treasury = fred.get_series('GS1')

In [10]:
# Retain only data since Jan 1959. 

data_1year_treasury = data_1year_treasury.loc[data_1year_treasury.index > '1958-12-31']

In [11]:
# View 1 year series

data_1year_treasury.head()

1959-01-01    3.36
1959-02-01    3.54
1959-03-01    3.61
1959-04-01    3.72
1959-05-01    3.96
dtype: float64

In [12]:
# Create DataFrame for each Series downloaded from Fred API

data_10year_treasury_df = pd.DataFrame(data_10year_treasury)

data_2year_treasury_df = pd.DataFrame(data_2year_treasury)

data_1year_treasury_df = pd.DataFrame(data_1year_treasury)


In [13]:
# Reset Index of 10 Year DataFrame

data_10year_treasury_df = data_10year_treasury_df.reset_index()

In [14]:
# View 10 Years DataFrame

data_10year_treasury_df.head()

,index,0
0,1959-01-01,4.02
1,1959-02-01,3.96
2,1959-03-01,3.99
3,1959-04-01,4.12
4,1959-05-01,4.31


In [15]:
# Reset Index of 2 Years DataFrame

data_2year_treasury_df = data_2year_treasury_df.reset_index()

In [16]:
# View 2 Years DataFrame

data_2year_treasury_df.head()

,index,0
0,1976-06-01,7.06
1,1976-07-01,6.85
2,1976-08-01,6.63
3,1976-09-01,6.42
4,1976-10-01,5.98


In [17]:
# Reset Index of 1 Year DataFrame

data_1year_treasury_df = data_1year_treasury_df.reset_index()

In [18]:
# View 1 year DataFrame 

data_1year_treasury_df.head()

,index,0
0,1959-01-01,3.36
1,1959-02-01,3.54
2,1959-03-01,3.61
3,1959-04-01,3.72
4,1959-05-01,3.96


In [19]:
# Merge 10 Year and 2 Year DataFrame to create Treasury DataFrame

treasury_df = pd.merge(data_10year_treasury_df,data_2year_treasury_df, on="index", how="left")

In [20]:
# Fill 0 for missing values

treasury_df = treasury_df.fillna(0)

In [21]:
# View the Treasury DataFrame

treasury_df.head()

,index,0_x,0_y
0,1959-01-01,4.02,0.0
1,1959-02-01,3.96,0.0
2,1959-03-01,3.99,0.0
3,1959-04-01,4.12,0.0
4,1959-05-01,4.31,0.0


In [22]:
# Merge Treasury DataFrame  and 1 Year Data Frame

treasury_df = pd.merge(treasury_df, data_1year_treasury_df, on="index")

In [23]:
# View the Treasury DataFrame

treasury_df.head()

,index,0_x,0_y,0
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [24]:
# Rename columns of Treasury DataFrame 

treasury_df = treasury_df.rename(columns={"index":"date", 
                                          "0_x":"10 Years T Rates", 
                                          "0_y": "2 Years T Rates",
                                         0 : "1 Year T Rates"})

In [25]:
# View the Treasury DataFrame

treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [26]:
# Create a function to fill missing 2 Year Rates with 10 Year rates.
# This will make the rate differnece zero for the period data for 2 year rates are missing

def adjust_2_year_missing_rates(twoYearValue, tenYearValue):
    if twoYearValue == 0:
        return tenYearValue
    else:
        return twoYearValue

In [27]:
# Adjust missing data for 2 year treasury rates

treasury_df["2 Years T Rates"] = treasury_df.apply(lambda row :  adjust_2_year_missing_rates(row["2 Years T Rates"], row["10 Years T Rates"]), axis=1)

In [28]:
# View the Treasury DataFrame

treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates
0,1959-01-01,4.02,4.02,3.36
1,1959-02-01,3.96,3.96,3.54
2,1959-03-01,3.99,3.99,3.61
3,1959-04-01,4.12,4.12,3.72
4,1959-05-01,4.31,4.31,3.96


In [29]:
# Create 10 - 2 years Rate Difference column

treasury_df["10-2 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["2 Years T Rates"]

In [30]:
# Create 10 - 1 year Rate Difference column

treasury_df["10-1 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["1 Year T Rates"]

In [31]:
# View the Treasury DataFrame

treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates,10-2 Rate Diff,10-1 Rate Diff
0,1959-01-01,4.02,4.02,3.36,0.0,0.66
1,1959-02-01,3.96,3.96,3.54,0.0,0.42
2,1959-03-01,3.99,3.99,3.61,0.0,0.38
3,1959-04-01,4.12,4.12,3.72,0.0,0.40
4,1959-05-01,4.31,4.31,3.96,0.0,0.35


In [32]:
# Print Data Types of columns
treasury_df.dtypes

date                datetime64[ns]
10 Years T Rates           float64
2 Years T Rates            float64
1 Year T Rates             float64
10-2 Rate Diff             float64
10-1 Rate Diff             float64
dtype: object

In [33]:
# Extract Quarter from date column

treasury_df['Year-Qtr'] = pd.PeriodIndex(treasury_df.date, freq='Q')

In [34]:
# Find mean after grouping by 'Year-Qtr'

treasury_df = treasury_df.groupby(['Year-Qtr']).mean()

In [35]:
# Drop columns that are no longer needed

treasury_df = treasury_df.drop(columns=["date", "10 Years T Rates", "2 Years T Rates", "1 Year T Rates"])

In [36]:
# Reset index 

treasury_df = treasury_df.reset_index()

In [37]:
# Rename column 

treasury_df = treasury_df.rename(columns={"index" : "Year-Qtr"})

In [38]:
# Cast Year-Qtr column to type of String

treasury_df["Year-Qtr"] = treasury_df["Year-Qtr"].astype("str")

In [39]:
# View the Treasury DataFrame

treasury_df.head()

,Year-Qtr,10-2 Rate Diff,10-1 Rate Diff
0,1959Q1,0.0,0.486667
1,1959Q2,0.0,0.340000
2,1959Q3,0.0,-0.100000
3,1959Q4,0.0,-0.333333
4,1960Q1,0.0,-0.083333


In [40]:
print("Treasury_Rates_Base_For_ML completed. treasury_df is ready for use ")

Treasury_Rates_Base_For_ML completed. treasury_df is ready for use 
